# SEGY 파일 처리 튜토리얼

이 노트북은 SEGY (SEG-Y) 형식의 지진 데이터를 로드, 분석, 분할, 시각화하는 방법을 보여줍니다.

## 목차
1. 환경 설정
2. SEGY 헤더 로드 및 분석
3. SEGY 데이터 로드
4. 데이터 분할
5. 데이터 시각화
6. 실제 사용 예제

## 1. 환경 설정

필요한 라이브러리를 설치하고 import합니다.

In [ ]:
# Google Colab에서 실행하는 경우
# 필요한 파일들을 업로드하거나 GitHub에서 다운로드

# 라이브러리 설치
!pip install segyio numpy matplotlib -q

print("라이브러리 설치 완료!")

In [ ]:
# 필요한 모듈 import
import numpy as np
import matplotlib.pyplot as plt
import segyio
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Colab에서 파일 업로드를 위한 import
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"Colab 환경: {IN_COLAB}")
print("모듈 import 완료!")

In [ ]:
# Python 모듈 파일 업로드 (Colab인 경우)
# header_loading.py, data_loading.py, data_divide.py 파일을 업로드하세요

if IN_COLAB:
    print("Python 모듈 파일을 업로드하세요: header_loading.py, data_loading.py, data_divide.py")
    uploaded = files.upload()
    print(f"업로드된 파일: {list(uploaded.keys())}")
else:
    print("로컬 환경에서 실행 중입니다.")

In [ ]:
# 모듈 import
from header_loading import SEGYHeaderLoader, load_segy_header
from data_loading import SEGYDataLoader, load_segy_data
from data_divide import SEGYDataDivider, divide_segy_file

print("커스텀 모듈 import 완료!")

## 2. SEGY 파일 업로드 및 헤더 분석

In [ ]:
# SEGY 파일 업로드
if IN_COLAB:
    print("SEGY 파일을 업로드하세요")
    uploaded = files.upload()
    segy_filename = list(uploaded.keys())[0]
else:
    # 로컬 환경에서는 파일 경로를 직접 지정
    segy_filename = "your_data.segy"  # 실제 파일 경로로 변경하세요

print(f"\n선택된 파일: {segy_filename}")

### 2.1 헤더 정보 전체 확인

In [ ]:
# SEGY 헤더 로드 및 분석
with SEGYHeaderLoader(segy_filename) as header_loader:
    # 전체 요약 정보 출력
    header_loader.print_header_summary()
    
    # 파일 정보 저장
    file_info = header_loader.get_file_info()

### 2.2 Textual Header 확인

In [ ]:
# Textual Header 출력
with SEGYHeaderLoader(segy_filename) as header_loader:
    header_loader.print_textual_header()

### 2.3 Trace Header 샘플 확인

In [ ]:
# 첫 번째 트레이스의 헤더 정보 확인
with SEGYHeaderLoader(segy_filename) as header_loader:
    header_loader.print_trace_header_sample(0)

### 2.4 주요 정보 요약

In [ ]:
# 주요 정보 변수로 저장
total_traces = file_info['total_traces']
samples_per_trace = file_info['samples_per_trace']
sample_interval_ms = file_info['sample_interval_ms']
total_time_ms = file_info['total_time_depth_ms']
data_size_mb = file_info['total_data_size_mb']

print("\n=== 데이터 요약 ===")
print(f"📊 총 트레이스 수: {total_traces:,}")
print(f"📏 트레이스당 샘플 수: {samples_per_trace:,}")
print(f"⏱️  샘플 간격: {sample_interval_ms} ms")
print(f"⏰ 총 시간/깊이: {total_time_ms:.2f} ms")
print(f"💾 전체 데이터 크기: {data_size_mb:.2f} MB")
print(f"📐 데이터 형태: ({total_traces}, {samples_per_trace})")

## 3. SEGY 데이터 로드

### 3.1 전체 데이터 정보 확인

In [ ]:
# 데이터 로더로 정보 확인
with SEGYDataLoader(segy_filename) as data_loader:
    data_loader.print_data_info(include_stats=True)

### 3.2 일부 데이터 로드 및 확인

In [ ]:
# 처음 100개 트레이스만 로드 (빠른 테스트용)
num_traces_to_load = min(100, total_traces)

with SEGYDataLoader(segy_filename) as data_loader:
    # 데이터 로드
    sample_data = data_loader.load_traces(0, num_traces_to_load)
    
    # 시간/깊이 축 생성
    time_axis = data_loader.get_time_axis()
    trace_axis = data_loader.get_trace_axis(0, num_traces_to_load)
    
    print(f"로드된 데이터 형태: {sample_data.shape}")
    print(f"시간 축 범위: {time_axis[0]:.2f} ~ {time_axis[-1]:.2f} ms")
    print(f"트레이스 축 범위: {trace_axis[0]} ~ {trace_axis[-1]}")

### 3.3 샘플 데이터 시각화

In [ ]:
# 간단한 시각화
plt.figure(figsize=(12, 6))
plt.imshow(sample_data.T, aspect='auto', cmap='seismic', 
           extent=[trace_axis[0], trace_axis[-1], time_axis[-1], time_axis[0]],
           vmin=-np.percentile(np.abs(sample_data), 95),
           vmax=np.percentile(np.abs(sample_data), 95))
plt.colorbar(label='Amplitude')
plt.xlabel('Trace Number')
plt.ylabel('Time/Depth (ms)')
plt.title(f'SEGY Data Preview (First {num_traces_to_load} Traces)')
plt.tight_layout()
plt.show()

## 4. 데이터 분할

대용량 SEGY 데이터를 처리하기 쉽도록 작은 청크로 분할합니다.

### 4.1 분할 파라미터 설정

In [ ]:
# 분할 파라미터 설정
traces_per_chunk = 100  # 청크당 트레이스 수
depth_interval_ms = 500.0  # 깊이/시간 간격 (ms)

print(f"분할 설정:")
print(f"  청크당 트레이스 수: {traces_per_chunk}")
print(f"  깊이/시간 간격: {depth_interval_ms} ms")

# 예상 청크 개수 계산
num_trace_chunks = (total_traces + traces_per_chunk - 1) // traces_per_chunk
num_depth_chunks = int((total_time_ms + depth_interval_ms - 1) // depth_interval_ms)
total_chunks = num_trace_chunks * num_depth_chunks

print(f"\n예상 청크 개수:")
print(f"  트레이스 방향: {num_trace_chunks}")
print(f"  깊이/시간 방향: {num_depth_chunks}")
print(f"  총 청크 수: {total_chunks}")

### 4.2 데이터 분할 실행

In [ ]:
# 데이터 분할 (저장하지 않고 정보만 확인)
with SEGYDataDivider(segy_filename) as divider:
    chunks = divider.divide_by_grid(traces_per_chunk, depth_interval_ms)
    divider.print_division_info(chunks)

### 4.3 특정 청크 데이터 추출 및 확인

In [ ]:
# 첫 번째 청크 추출
chunk_idx = 0
selected_chunk = chunks[chunk_idx]

with SEGYDataDivider(segy_filename) as divider:
    chunk_data = divider.extract_chunk(
        selected_chunk['trace_range'],
        selected_chunk['sample_range']
    )

print(f"청크 #{chunk_idx} 정보:")
print(f"  트레이스 범위: {selected_chunk['trace_range']}")
print(f"  샘플 범위: {selected_chunk['sample_range']}")
print(f"  시간 범위: {selected_chunk['time_range_ms']} ms")
print(f"  데이터 형태: {chunk_data.shape}")
print(f"  데이터 통계:")
print(f"    최소값: {chunk_data.min():.6e}")
print(f"    최대값: {chunk_data.max():.6e}")
print(f"    평균: {chunk_data.mean():.6e}")
print(f"    표준편차: {chunk_data.std():.6e}")

### 4.4 청크 데이터 시각화

In [ ]:
# 청크 시각화
t_start, t_end = selected_chunk['trace_range']
time_start, time_end = selected_chunk['time_range_ms']

plt.figure(figsize=(12, 6))
plt.imshow(chunk_data.T, aspect='auto', cmap='seismic',
           extent=[t_start, t_end, time_end, time_start],
           vmin=-np.percentile(np.abs(chunk_data), 95),
           vmax=np.percentile(np.abs(chunk_data), 95))
plt.colorbar(label='Amplitude')
plt.xlabel('Trace Number')
plt.ylabel('Time/Depth (ms)')
plt.title(f'Chunk #{chunk_idx} Visualization')
plt.tight_layout()
plt.show()

## 5. 고급 시각화

### 5.1 여러 청크 비교 시각화

In [ ]:
# 처음 4개 청크 시각화
num_chunks_to_show = min(4, len(chunks))

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

with SEGYDataDivider(segy_filename) as divider:
    for i in range(num_chunks_to_show):
        chunk = chunks[i]
        chunk_data = divider.extract_chunk(
            chunk['trace_range'],
            chunk['sample_range']
        )
        
        t_start, t_end = chunk['trace_range']
        time_start, time_end = chunk['time_range_ms']
        
        im = axes[i].imshow(chunk_data.T, aspect='auto', cmap='seismic',
                           extent=[t_start, t_end, time_end, time_start],
                           vmin=-np.percentile(np.abs(chunk_data), 95),
                           vmax=np.percentile(np.abs(chunk_data), 95))
        axes[i].set_xlabel('Trace Number')
        axes[i].set_ylabel('Time/Depth (ms)')
        axes[i].set_title(f'Chunk #{i}')
        plt.colorbar(im, ax=axes[i], label='Amplitude')

plt.tight_layout()
plt.show()

### 5.2 단일 트레이스 시각화

In [ ]:
# 여러 트레이스를 개별적으로 플롯
traces_to_plot = [0, total_traces // 4, total_traces // 2, total_traces * 3 // 4]
traces_to_plot = [t for t in traces_to_plot if t < total_traces]

fig, axes = plt.subplots(1, len(traces_to_plot), figsize=(16, 6), sharey=True)

with SEGYDataLoader(segy_filename) as loader:
    time_axis = loader.get_time_axis()
    
    for i, trace_idx in enumerate(traces_to_plot):
        trace_data = loader.load_trace(trace_idx)
        
        axes[i].plot(trace_data, time_axis, 'b-', linewidth=0.5)
        axes[i].fill_betweenx(time_axis, 0, trace_data, where=(trace_data > 0), 
                             color='red', alpha=0.5)
        axes[i].fill_betweenx(time_axis, 0, trace_data, where=(trace_data < 0), 
                             color='blue', alpha=0.5)
        axes[i].set_xlabel('Amplitude')
        axes[i].set_title(f'Trace #{trace_idx}')
        axes[i].grid(True, alpha=0.3)
        axes[i].invert_yaxis()

axes[0].set_ylabel('Time/Depth (ms)')
plt.tight_layout()
plt.show()

### 5.3 진폭 스펙트럼 분석

In [ ]:
# 진폭 히스토그램
with SEGYDataLoader(segy_filename) as loader:
    # 샘플링하여 로드 (전체 로드는 메모리 부족 가능)
    sample_traces = min(1000, total_traces)
    sample_data = loader.load_traces(0, sample_traces)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# 히스토그램
axes[0].hist(sample_data.flatten(), bins=100, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Amplitude')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Amplitude Distribution')
axes[0].grid(True, alpha=0.3)

# 박스 플롯
axes[1].boxplot([sample_data.flatten()], vert=True)
axes[1].set_ylabel('Amplitude')
axes[1].set_title('Amplitude Box Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 통계 정보
print("\n진폭 통계:")
print(f"  최소값: {sample_data.min():.6e}")
print(f"  최대값: {sample_data.max():.6e}")
print(f"  평균: {sample_data.mean():.6e}")
print(f"  표준편차: {sample_data.std():.6e}")
print(f"  중앙값: {np.median(sample_data):.6e}")
print(f"  5th percentile: {np.percentile(sample_data, 5):.6e}")
print(f"  95th percentile: {np.percentile(sample_data, 95):.6e}")

## 6. 청크 저장 (옵션)

In [ ]:
# 청크를 파일로 저장하려면 실행
# 주의: 청크 개수가 많으면 시간이 오래 걸릴 수 있습니다

SAVE_CHUNKS = False  # True로 변경하면 저장

if SAVE_CHUNKS:
    output_dir = "./segy_chunks"
    
    with SEGYDataDivider(segy_filename) as divider:
        divider.save_all_chunks(chunks, output_dir, prefix="chunk")
    
    print(f"\n청크가 '{output_dir}' 디렉토리에 저장되었습니다.")
    
    # Colab에서 다운로드
    if IN_COLAB:
        # ZIP으로 압축하여 다운로드
        import shutil
        shutil.make_archive('segy_chunks', 'zip', output_dir)
        files.download('segy_chunks.zip')
else:
    print("청크 저장이 비활성화되어 있습니다. SAVE_CHUNKS = True로 설정하여 저장하세요.")

## 7. 사용자 정의 분할 및 시각화

In [ ]:
# 사용자가 원하는 범위를 지정하여 데이터 추출 및 시각화

# 파라미터 설정
custom_trace_start = 0
custom_trace_end = min(200, total_traces)
custom_time_start_ms = 0
custom_time_end_ms = 1000  # 1초

# 시간을 샘플 인덱스로 변환
custom_sample_start = int(custom_time_start_ms / sample_interval_ms)
custom_sample_end = int(custom_time_end_ms / sample_interval_ms)

print(f"사용자 정의 범위:")
print(f"  트레이스: {custom_trace_start} ~ {custom_trace_end}")
print(f"  시간: {custom_time_start_ms} ~ {custom_time_end_ms} ms")
print(f"  샘플: {custom_sample_start} ~ {custom_sample_end}")

# 데이터 로드
custom_data, metadata = load_segy_data(
    segy_filename,
    start_trace=custom_trace_start,
    end_trace=custom_trace_end,
    start_sample=custom_sample_start,
    end_sample=custom_sample_end
)

print(f"\n로드된 데이터 형태: {custom_data.shape}")

# 시각화
plt.figure(figsize=(14, 8))
plt.imshow(custom_data.T, aspect='auto', cmap='seismic',
           extent=[custom_trace_start, custom_trace_end, 
                   custom_time_end_ms, custom_time_start_ms],
           vmin=-np.percentile(np.abs(custom_data), 95),
           vmax=np.percentile(np.abs(custom_data), 95))
plt.colorbar(label='Amplitude', shrink=0.8)
plt.xlabel('Trace Number', fontsize=12)
plt.ylabel('Time/Depth (ms)', fontsize=12)
plt.title(f'Custom Range Visualization\nTraces: {custom_trace_start}-{custom_trace_end}, '
          f'Time: {custom_time_start_ms}-{custom_time_end_ms} ms', fontsize=14)
plt.tight_layout()
plt.show()

## 8. 요약 및 정리

In [ ]:
print("="*70)
print("SEGY 파일 처리 완료 요약")
print("="*70)
print(f"\n📁 파일: {segy_filename}")
print(f"\n📊 데이터 정보:")
print(f"   - 총 트레이스: {total_traces:,}")
print(f"   - 샘플/트레이스: {samples_per_trace:,}")
print(f"   - 샘플 간격: {sample_interval_ms} ms")
print(f"   - 총 시간/깊이: {total_time_ms:.2f} ms")
print(f"   - 데이터 크기: {data_size_mb:.2f} MB")
print(f"\n📦 분할 정보:")
print(f"   - 청크당 트레이스: {traces_per_chunk}")
print(f"   - 깊이/시간 간격: {depth_interval_ms} ms")
print(f"   - 총 청크 수: {len(chunks)}")
print(f"\n✅ 처리 완료!")
print("="*70)

## 참고사항

### 주요 기능
1. **헤더 분석**: Textual, Binary, Trace 헤더 정보 추출
2. **데이터 로드**: 전체 또는 부분 데이터 로드
3. **데이터 분할**: 트레이스 및 깊이/시간 기준 분할
4. **시각화**: 다양한 형태의 데이터 시각화

### 사용 팁
- 대용량 파일의 경우 전체 로드 대신 부분 로드를 사용하세요
- 청크 크기는 메모리와 처리 속도를 고려하여 조정하세요
- 시각화 시 percentile을 사용하여 극값의 영향을 줄이세요

### 추가 작업
- 청크를 NumPy 파일로 저장하여 나중에 빠르게 로드
- 각 청크에 대한 처리 및 분석 수행
- 머신러닝 모델 학습을 위한 데이터 전처리

### 문의 및 지원
문제가 발생하면 각 모듈의 docstring을 참고하거나 이슈를 등록하세요.